### Start MySQL in Docker

In [5]:
!docker run --name python101-mysql -e MYSQL_ROOT_PASSWORD=supersecret -e MYSQL_DATABASE=python101 -p 3306:3306 -d mysql:latest

16103c86ba278e40d8dc7e4c20f5dc67b38220a9af7ad2fb4828f579a64354c8


Unable to find image 'mysql:latest' locally
latest: Pulling from library/mysql
4a629f1008ff: Pulling fs layer
00af5f73db53: Pulling fs layer
8314ff001dec: Pulling fs layer
61941d436e88: Pulling fs layer
658630c937c6: Pulling fs layer
74a8e4bbd9fe: Pulling fs layer
f3702ac323ca: Pulling fs layer
c89a19cdad3b: Pulling fs layer
357926362d31: Pulling fs layer
ab1b8ea72826: Pulling fs layer
00af5f73db53: Download complete
4a629f1008ff: Download complete
357926362d31: Download complete
ab1b8ea72826: Download complete
61941d436e88: Download complete
f3702ac323ca: Download complete
658630c937c6: Download complete
74a8e4bbd9fe: Download complete
c89a19cdad3b: Download complete
74a8e4bbd9fe: Pull complete
357926362d31: Pull complete
f3702ac323ca: Pull complete
8314ff001dec: Download complete
658630c937c6: Pull complete
ab1b8ea72826: Pull complete
61941d436e88: Pull complete
00af5f73db53: Pull complete
c89a19cdad3b: Pull complete
8314ff001dec: Pull complete
4a629f1008ff: Pull complete
Digest: sha

Install Dependencies

In [19]:
%pip install pandas requests mysql-connector-python sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\tombo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### DB Config

In [20]:
from sqlalchemy import create_engine

# --- Beállítások ---
DB_HOST = "127.0.0.1"
DB_PORT = 3306
DB_USER = "root"
DB_PASSWORD = "supersecret"
DB_NAME = "python101"
TABLE_NAME = "tombor_payloads"
URL = "https://api.spacexdata.com/v3/launches"

# Közös SQLAlchemy Engine létrehozása az összes művelethez
connection_string = f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)

print("Konfiguráció betöltve, adatbázis motor (engine) készen áll.")

Konfiguráció betöltve, adatbázis motor (engine) készen áll.


### Korábbi DB Törlése (teszteléshez)

In [21]:
from sqlalchemy import text

print(f"A(z) '{TABLE_NAME}' tábla törlése folyamatban...")

try:
    with engine.begin() as conn:
        conn.execute(text(f"DROP TABLE IF EXISTS {TABLE_NAME};"))
        
    print(f"KÉSZ! A(z) '{TABLE_NAME}' tábla sikeresen törölve lett (amennyiben létezett).")

except Exception as e:
    print(f"Hiba történt: {e}")
finally:
    engine.dispose()
    print("Adatbázis motor (engine) leállítva, kapcsolatok lezárva.")

A(z) 'tombor_payloads' tábla törlése folyamatban...
KÉSZ! A(z) 'tombor_payloads' tábla sikeresen törölve lett (amennyiben létezett).
Adatbázis motor (engine) leállítva, kapcsolatok lezárva.


### DB feltöltése API-ból

In [22]:
import requests
from datetime import datetime
from sqlalchemy import text

print("1. Adatok letöltése a SpaceX API-ról...")
try:
    response = requests.get(URL)
    response.raise_for_status()
    launches = response.json()
    print("Sikeres letöltés!\n")
    
    print(f"2. '{TABLE_NAME}' tábla létrehozása és adatok betöltése...")
    with engine.begin() as conn:
        conn.execute(text(f"""
            CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
                id INT AUTO_INCREMENT PRIMARY KEY,
                launch_date_utc DATETIME,
                launch_success BOOLEAN,
                payload_id VARCHAR(255),
                orbit VARCHAR(255)
            )
        """))

        insert_query = text(f"""
            INSERT INTO {TABLE_NAME} (launch_date_utc, launch_success, payload_id, orbit)
            VALUES (:launch_date_utc, :launch_success, :payload_id, :orbit)
        """)

        inserted_count = 0
        
        for launch in launches:
            raw_date = launch.get("launch_date_utc")
            if raw_date:
                # API formátuma: '2006-03-24T22:30:00.000Z'
                parsed_date = datetime.strptime(raw_date, "%Y-%m-%dT%H:%M:%S.%fZ")
                launch_date_utc = parsed_date.strftime("%Y-%m-%d %H:%M:%S")
            else:
                launch_date_utc = None
            
            launch_success = launch.get("launch_success")
            
            rocket = launch.get("rocket", {})
            second_stage = rocket.get("second_stage", {})
            payloads = second_stage.get("payloads", [])
            
            for payload in payloads:
                payload_id = payload.get("payload_id")
                orbit = payload.get("orbit")
                
                conn.execute(insert_query, {
                    "launch_date_utc": launch_date_utc,
                    "launch_success": launch_success,
                    "payload_id": payload_id,
                    "orbit": orbit
                })
                inserted_count += 1

    print(f"KÉSZ! Sikeresen beillesztve {inserted_count} hasznos teher (payload) rekord a(z) '{TABLE_NAME}' táblába.")

except Exception as e:
    print(f"Hiba történt: {e}")

1. Adatok letöltése a SpaceX API-ról...
Sikeres letöltés!

2. 'tombor_payloads' tábla létrehozása és adatok betöltése...
KÉSZ! Sikeresen beillesztve 136 hasznos teher (payload) rekord a(z) 'tombor_payloads' táblába.


### Keresés időintervallum alapján

In [23]:
import pandas as pd
from datetime import datetime
from IPython.display import display
from sqlalchemy import text

start_date_str = input("Kérlek, add meg a kezdő dátumot (ÉÉÉÉ-HH-NN) [Alapértelmezett: 2018-01-01]: ").strip() or "2018-01-01"
end_date_str = input("Kérlek, add meg a végdátumot (ÉÉÉÉ-HH-NN) [Alapértelmezett: 2018-12-31]: ").strip() or "2018-12-31"

try:
    print(f"Keresés időintervallum alapján ({start_date_str} - {end_date_str})\n")
    
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d").replace(hour=23, minute=59, second=59)
    
    query_interval = text(f"""
        SELECT launch_date_utc AS 'Dátum', 
               IF(launch_success = 1, 'Sikeres', IF(launch_success = 0, 'Sikertelen', 'Ismeretlen')) AS 'Eredmény', 
               payload_id AS 'Payload ID', 
               orbit AS 'Pálya'
        FROM {TABLE_NAME}
        WHERE launch_date_utc >= :start_date AND launch_date_utc <= :end_date
        ORDER BY launch_date_utc;
    """)
    
    with engine.connect() as conn:
        df_interval = pd.read_sql(
            query_interval, 
            conn, 
            params={"start_date": start_date, "end_date": end_date}
        )
    
    if not df_interval.empty:
        print(f"Találatok száma: {len(df_interval)}")
        display(df_interval)
    else:
        print("Nincs találat a megadott intervallumban.")

except ValueError:
    print("Hibás dátumformátum! Kérlek, használd a megadott ÉÉÉÉ-HH-NN formátumot.")
except Exception as e:
    print(f"Hiba történt: {e}")

Keresés időintervallum alapján (2018-01-01 - 2018-12-31)

Találatok száma: 23


,Dátum,Eredmény,Payload ID,Pálya
0,2018-01-08 01:00:00,Sikeres,ZUMA,LEO
1,2018-01-31 21:25:00,Sikeres,GovSat-1,GTO
2,2018-02-06 20:45:00,Sikeres,Tesla Roadster,HCO
3,2018-02-22 14:17:00,Sikeres,Paz,SSO
4,2018-02-22 14:17:00,Sikeres,Tintin A & B,SSO
5,2018-03-06 05:33:00,Sikeres,Hispasat 30W-6,GTO
6,2018-03-30 14:13:51,Sikeres,Iridium NEXT 5,PO
7,2018-04-02 20:30:41,Sikeres,CRS-14,ISS
8,2018-04-18 22:51:00,Sikeres,TESS,HEO
9,2018-05-11 20:14:00,Sikeres,Bangabandhu-1,GTO


### Több pályás (multi-orbit) fellövések adatai

In [24]:
import pandas as pd
from IPython.display import display
from sqlalchemy import text

try:
    print("Több pályás (multi-orbit) fellövések adatai\n")
    
    query_multi_orbit = text(f"""
        SELECT launch_date_utc AS 'Dátum', 
               IF(launch_success = 1, 'Sikeres', IF(launch_success = 0, 'Sikertelen', 'Ismeretlen')) AS 'Eredmény', 
               payload_id AS 'Payload ID', 
               orbit AS 'Pálya'
        FROM {TABLE_NAME}
        WHERE launch_date_utc IN (
            SELECT launch_date_utc
            FROM {TABLE_NAME}
            GROUP BY launch_date_utc
            HAVING COUNT(DISTINCT orbit) > 1
        )
        ORDER BY launch_date_utc, payload_id;
    """)
    
    with engine.connect() as conn:
        df_multi = pd.read_sql(query_multi_orbit, conn)
    
    if not df_multi.empty:
        print(f"Összesen {len(df_multi)} db payload tartozik ilyen fellövéshez:")
        display(df_multi)
    else:
        print("Nem található olyan fellövés, ahol egyszerre több különböző pályára lőttek volna fel objektumokat.")

except Exception as e:
    print(f"Hiba történt: {e}")

Több pályás (multi-orbit) fellövések adatai

Összesen 20 db payload tartozik ilyen fellövéshez:


,Dátum,Eredmény,Payload ID,Pálya
0,2012-10-08 00:35:00,Sikeres,CRS-1,ISS
1,2012-10-08 00:35:00,Sikeres,Orbcomm-OG2,LEO
2,2019-06-25 03:30:00,Sikeres,ARMADILLO,LEO
3,2019-06-25 03:30:00,Sikeres,BRICSat 2,LEO
4,2019-06-25 03:30:00,Sikeres,COSMIC-2,LEO
5,2019-06-25 03:30:00,Sikeres,CP9,LEO
6,2019-06-25 03:30:00,Sikeres,DSX,MEO
7,2019-06-25 03:30:00,Sikeres,E-TBEx A & B,LEO
8,2019-06-25 03:30:00,Sikeres,FalconSat 7,LEO
9,2019-06-25 03:30:00,Sikeres,GPIM,LEO
